In [79]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/final-data/__results__.html
/kaggle/input/final-data/__notebook__.ipynb
/kaggle/input/final-data/__output__.json
/kaggle/input/final-data/custom.css
/kaggle/input/final-data/phase2/user_train_phase_2_test.csv
/kaggle/input/final-data/phase2/user_train_phase_2.csv
/kaggle/input/final-data/phase2/user_train_phase_2_train.csv
/kaggle/input/final-data/__results___files/__results___144_0.png
/kaggle/input/final-data/__results___files/__results___76_0.png
/kaggle/input/final-data/__results___files/__results___149_0.png
/kaggle/input/final-data/__results___files/__results___31_0.png
/kaggle/input/final-data/__results___files/__results___66_0.png
/kaggle/input/final-data/__results___files/__results___59_0.png
/kaggle/input/final-data/__results___files/__results___69_0.png
/kaggle/input/final-data/__results___files/__results___27_0.png
/kaggle/input/final-data/__results___files/__results___81_0.png
/kaggle/input/final-data/__results___files/__results___71_0.png
/kaggle/input/final

In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import (f1_score, precision_score, recall_score, accuracy_score, 
                             roc_auc_score, roc_curve, confusion_matrix)
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.preprocessing import label_binarize
from sklearn.linear_model import LassoCV
from sklearn.decomposition import PCA
from sklearn.calibration import CalibratedClassifierCV  
from eli5.sklearn import PermutationImportance  

from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import (
    f1_score, precision_score, recall_score, accuracy_score,
    roc_auc_score, confusion_matrix, roc_curve
)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from catboost import CatBoostClassifier

from tensorflow.keras.utils import to_categorical

import warnings
warnings.filterwarnings("ignore")

# 1. Load dữ liệu

In [81]:
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical


def load_data(phase, timesteps, one_hot=True):
    """
    Đọc dữ liệu, tiền xử lý và reshape cho LSTM.
    Trả về X_train, y_train, X_test, y_test với X có shape (n_samples, timesteps, n_features).
    Không thực hiện scaling.
    """
    # Định dạng lại tên phase để khớp file
    phase_ = phase[:-1] + '_' + phase[-1]
    train_fp = f"/kaggle/input/final-data/{phase}/user_train_{phase_}_train.csv"
    test_fp  = f"/kaggle/input/final-data/{phase}/user_train_{phase_}_test.csv"

    # Đọc dữ liệu
    df_train = pd.read_csv(train_fp)
    df_test  = pd.read_csv(test_fp)

    # Mã hoá school thành tần suất
    freq_map = df_train['school'].value_counts().to_dict()
    df_train['school'] = df_train['school'].map(freq_map)
    df_test ['school'] = df_test ['school'].map(freq_map)

    # Fill missing
    df_train = df_train.fillna(0)
    df_test  = df_test.fillna(0)

    # Tách nhãn
    y_train = df_train['label_encoded'].values
    y_test  = df_test ['label_encoded'].values

    # Loại bỏ các cột không cần thiết
    drop_cols = ['course_id', 'user_id', 'total_score', 'label', 'label_encoded']
    X_train_df = df_train.drop(columns=drop_cols)
    X_test_df  = df_test .drop(columns=drop_cols)

    # Chuyển DataFrame thành mảng numpy
    X_train = X_train_df.values
    X_test  = X_test_df.values

    # Cắt cho chia hết cho timesteps
    n_train = (X_train.shape[0] // timesteps) * timesteps
    n_test  = (X_test.shape[0]  // timesteps) * timesteps
    X_train = X_train[:n_train]
    y_train = y_train[:n_train]
    X_test  = X_test[:n_test]
    y_test  = y_test[:n_test]


    if one_hot:
        num_classes = len(np.unique(y_train))
        if num_classes > 2:
            y_train = to_categorical(y_train, num_classes=num_classes)
            y_test  = to_categorical(y_test,  num_classes=num_classes)
        # Với bài toán nhị phân thì có thể để nguyên, hoặc vẫn dùng one-hot nếu muốn
    # Reshape thành (n_samples, timesteps, n_features)
    # n_features = X_train.shape[1]
    # X_train = X_train.reshape(-1, timesteps, n_features)
    # X_test  = X_test.reshape(-1, timesteps, n_features)

    return X_train, y_train, X_test, y_test


# 2. Đánh giá dữ liệu

In [82]:
def evaluate_model(model, X_train, y_train, X_test, y_test, class_names=None):
    """
    Đánh giá model trên train và test, vẽ confusion matrix và ROC.
    Trả về train_results và test_results giống cấu trúc ban đầu.
    """
    # Xác định classes và tên lớp
    classes = np.unique(np.concatenate([y_train, y_test]))
    if class_names is None:
        class_names = [str(c) for c in classes]

    # Hàm lấy xác suất dự đoán
    def get_proba(X):
        proba = model.predict(X)
        if proba.ndim == 1:  # binary case
            proba = np.vstack([1 - proba, proba]).T
        return proba

    # Hàm đánh giá từng split
    def eval_split(X, y, name):
        y_proba = get_proba(X)
        y_pred = np.argmax(y_proba, axis=1)

        f1 = f1_score(y, y_pred, average=None, labels=classes)
        prec = precision_score(y, y_pred, average=None, labels=classes)
        rec  = recall_score(y, y_pred, average=None, labels=classes)
        acc  = accuracy_score(y, y_pred)

        y_bin = label_binarize(y, classes=classes)
        auc = roc_auc_score(y_bin, y_proba, average='macro', multi_class='ovr')

        print(f"\n--- {name} Set ---")
        for idx, cname in enumerate(class_names):
            print(f"{cname}: Prec={prec[idx]:.3f}, Rec={rec[idx]:.3f}, F1={f1[idx]:.3f}")
        print(f"Accuracy: {acc:.3f}, AUC: {auc:.3f}")

        return {
            'f1': f1,
            'precision': prec,
            'recall': rec,
            'accuracy': acc,
            'auc': auc,
            'y_true': y,
            'y_pred': y_pred,
            'y_proba': y_proba
        }

    # Đánh giá train và test
    train_results = eval_split(X_train, y_train, 'Train')
    test_results  = eval_split(X_test,  y_test,  'Test')

    # Confusion matrix cho Test
    cm = confusion_matrix(test_results['y_true'], test_results['y_pred'], labels=classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix (Test)')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.tight_layout()
    plt.show()

    # ROC curves cho mỗi lớp (Test)
    y_te_bin = label_binarize(test_results['y_true'], classes=classes)
    plt.figure(figsize=(8, 6))
    for idx, cname in enumerate(class_names):
        fpr, tpr, _ = roc_curve(y_te_bin[:, idx], test_results['y_proba'][:, idx])
        auc_score = roc_auc_score(y_te_bin[:, idx], test_results['y_proba'][:, idx])
        plt.plot(fpr, tpr, label=f"{cname} (AUC={auc_score:.2f})")
    plt.plot([0, 1], [0, 1], 'k--', label='Chance')
    plt.title('ROC Curves (Test)')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='best')
    plt.tight_layout()
    plt.show()

    return {
        'train': train_results,
        'test': test_results
    }


# 3. Scale dữ liệu

In [83]:
# Áp dụng các kỹ thuật Scaling
def scale_data(X_train, X_test, scaler_type):
    # Khởi tạo bộ scaler phù hợp với loại được chọn
    if scaler_type == "minmax":
        scaler = MinMaxScaler()
    elif scaler_type == "standard":
        scaler = StandardScaler()
    elif scaler_type == "robust":
        scaler = RobustScaler()
    elif scaler_type == "log":
        # Thực hiện log scaling, cộng thêm một giá trị nhỏ để tránh log(0)
        if (X_train <= 0).any():
            X_train = X_train + 1 - X_train.min()  # Điều chỉnh để tất cả giá trị > 0
        X_train = np.log(X_train)  # Áp dụng log cho dữ liệu
        X_test = np.log(X_test + 1 - X_test.min())  # Áp dụng log cho dữ liệu test
        return X_train, X_test
    else:
        raise ValueError("Scaler type not recognized")
    
    # Fit bộ scaler trên dữ liệu huấn luyện và transform dữ liệu huấn luyện và kiểm tra
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)  # Transform dữ liệu test bằng scaler đã fit trên train
    
    return X_train_scaled, X_test_scaled

# 4. Permutation importance

In [84]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.inspection import permutation_importance

def plot_permutation_importance(model, X_train, y_train, feature_names=None, use='last'):
    """
    Tính và vẽ permutation importance với model và X_train (2D hoặc 3D).
    - Nếu X_train 3D: lấy bước thời gian cuối cùng (hoặc mean) để chuyển về 2D.
    - model phải là model đã fit và hỗ trợ predict trên input 2D.
    """
    # Nếu X_train là 3D: (n_samples, timesteps, n_features)
    if len(X_train.shape) == 3:
        if use == 'last':
            X_tabular = X_train[:, -1, :]
        elif use == 'mean':
            X_tabular = X_train.mean(axis=1)
        else:
            raise ValueError("Giá trị use phải là 'last' hoặc 'mean'.")
    else:
        X_tabular = X_train

    # Nếu chưa có feature_names, tự tạo
    if feature_names is None:
        feature_names = [f'F{i}' for i in range(X_tabular.shape[1])]

    # Nếu X_tabular không phải DataFrame, chuyển về DataFrame để dùng .columns
    X_df = pd.DataFrame(X_tabular, columns=feature_names)

    # Nếu model là keras (Sequential), phải wrap lại để trả ra dự đoán xác suất class cao nhất
    from sklearn.base import BaseEstimator, ClassifierMixin
    import numpy as np

    class KerasWrapper(BaseEstimator, ClassifierMixin):
        def __init__(self, model):
            self.model = model
        def fit(self, X, y):
            # Không dùng fit, chỉ để tương thích API
            return self
        def predict(self, X):
            proba = self.model.predict(X)
            if proba.shape[1] == 1:
                return (proba > 0.5).astype(int).flatten()
            else:
                return np.argmax(proba, axis=1)

    # Nếu model là Keras (Sequential), wrap lại
    if "keras" in str(type(model)).lower() or "tensorflow" in str(type(model)).lower():
        model = KerasWrapper(model)

    # Chạy permutation importance
    result = permutation_importance(model, X_df, y_train, n_repeats=10, random_state=42, n_jobs=-1)
    importance_df = pd.DataFrame({
        "Feature": X_df.columns,
        "Importance": result.importances_mean
    }).sort_values(by="Importance", ascending=False)
    
    # Vẽ biểu đồ Importance
    plt.figure(figsize=(10, 6))
    sns.barplot(x="Importance", y="Feature", data=importance_df)
    plt.title("Feature Importance (Permutation Importance)")
    plt.xlabel("Importance")
    plt.ylabel("Feature")
    plt.tight_layout()
    plt.show()
    
    return importance_df


# Lasso

In [85]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.linear_model import LassoCV

def lasso_feature_selection(X_train, y_train, feature_names=None, use='last'):
    """
    - X_train: array 2D hoặc 3D (nếu 3D sẽ lấy bước cuối cùng hoặc mean)
    - y_train: label (1D)
    - feature_names: tên cột (nếu None sẽ tự tạo)
    - use: 'last' (bước cuối) hoặc 'mean' (trung bình trên time)
    """
    # Chuyển từ 3D về 2D nếu cần
    if len(X_train.shape) == 3:
        if use == 'last':
            X_tabular = X_train[:, -1, :]
        elif use == 'mean':
            X_tabular = X_train.mean(axis=1)
        else:
            raise ValueError("use must be 'last' hoặc 'mean'")
    else:
        X_tabular = X_train

    # Đảm bảo X_tabular là DataFrame
    if feature_names is None:
        feature_names = [f'F{i}' for i in range(X_tabular.shape[1])]
    X_df = pd.DataFrame(X_tabular, columns=feature_names)

    # Lasso Regression với Cross-Validation
    lasso = LassoCV(cv=5, random_state=42, n_jobs=-1)
    lasso.fit(X_df, y_train)

    feature_importance = pd.DataFrame({
        'Feature': X_df.columns,
        'Importance': np.abs(lasso.coef_)
    }).sort_values(by='Importance', ascending=False)
    
    # Vẽ biểu đồ trọng số
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Importance', y='Feature', data=feature_importance)
    plt.title("Feature Importance (Lasso Regression)")
    plt.xlabel("Importance")
    plt.ylabel("Feature")
    plt.tight_layout()
    plt.show()
    
    selected_features = feature_importance[feature_importance['Importance'] > 0]['Feature'].tolist()

    return selected_features, feature_importance


# Boruta Truck

In [86]:
def boruta_trick(model, X_train, y_train):
    # Tạo một feature ngẫu nhiên (feature "ngẫu nhiên")
    random_feature = np.random.rand(X_train.shape[0], 1)
    X_train_random = np.hstack([X_train.values, random_feature])

    # Huấn luyện mô hình Random Forest
    model.fit(X_train_random, y_train)

    # Tính toán importance của từng feature
    feature_importance = pd.DataFrame({
        'Feature': list(X_train.columns) + ['Random Feature'],
        'Importance': model.feature_importances_
    }).sort_values(by='Importance', ascending=False)

    # Lọc các feature quan trọng hơn feature ngẫu nhiên
    important_features = feature_importance[feature_importance['Importance'] > feature_importance['Importance'].iloc[-1]]
    print('important feature: ', important_features)
    # Vẽ biểu đồ importance
    feature_importance.plot(kind='barh', x='Feature', y='Importance', legend=False, figsize=(8, 6))
    plt.title("Boruta Feature Importance (Trick)")
    plt.xlabel("Importance")
    plt.ylabel("Feature")
    plt.show()

    return important_features

# Calibration

In [ ]:
def calibrate_model(model, X_train, y_train, method='sigmoid'):
    """
    Áp dụng Calibration cho mô hình phân loại (hỗ trợ CatBoost và các mô hình khác).
    
    Parameters:
    - model: Mô hình đã huấn luyện
    - X_train: Dữ liệu huấn luyện
    - y_train: Nhãn huấn luyện
    - method: Phương pháp calibration ('sigmoid' hoặc 'isotonic')
    
    Returns:
    - calibrated_model: Mô hình đã được calibrate
    """
    # Nếu là CatBoost, thông báo nhẹ
    if isinstance(model, CatBoostClassifier):
        print("[Warning] CatBoostClassifier đã hỗ trợ predict_proba tốt. Calibration có thể không cần thiết.")
    
    # Calibration
    calibrated_model = CalibratedClassifierCV(base_estimator=model, method=method, cv=5)
    calibrated_model.fit(X_train, y_train)
    
    return calibrated_model

# 1.Standard

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical


def build_lstm_model(input_shape, units=64, dropout_rate=0.2, num_classes=1):
    model = Sequential()
    model.add(LSTM(units, input_shape=input_shape, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='sigmoid' if num_classes == 1 else 'softmax'))
    model.compile(loss='binary_crossentropy' if num_classes == 1 else 'categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

def process_phase(phase, scale='standard', metric='accuracy',timesteps=1):
    # Đọc dữ liệu
    X_train, y_train, X_test, y_test = load_data(phase, timesteps,one_hot=False)

    # if num_classes > 2:
    #     y_train_cat = to_categorical(y_train, num_classes=num_classes)
    # else:
    #     y_train_cat = y_train
    # Scale dữ liệu
    X_train_scaled, X_test_scaled = scale_data(X_train, X_test, scaler_type=scale)
    
    # Đảm bảo đúng shape cho LSTM: (samples, timesteps, features)
    if len(X_train_scaled.shape) == 2:
        # reshape nếu đầu vào chưa đủ 3 chiều
        X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, X_train_scaled.shape[1]))
        X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], timesteps, X_test_scaled.shape[1]))

    input_shape = (X_train_scaled.shape[1], X_train_scaled.shape[2])
    num_classes = 1 if len(np.unique(y_train)) == 2 else len(np.unique(y_train))

    # Thiết lập StratifiedKFold
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    best_score = -np.inf
    best_params = {}
    best_model = None

    # Grid search thủ công (do Keras không hợp với RandomizedSearchCV)
    for units in [32, 64]:
        for dropout_rate in [0.1, 0.2]:
            fold_scores = []
            for train_idx, val_idx in skf.split(X_train_scaled, y_train):
                X_tr, X_val = X_train_scaled[train_idx], X_train_scaled[val_idx]
                y_tr, y_val = y_train[train_idx], y_train[val_idx]
            
                if num_classes > 2:
                    y_tr_cat = to_categorical(y_tr, num_classes=num_classes)
                    y_val_cat = to_categorical(y_val, num_classes=num_classes)
                else:
                    y_tr_cat = y_tr
                    y_val_cat = y_val
            
                model = build_lstm_model(input_shape, units=units, dropout_rate=dropout_rate, num_classes=num_classes)
                es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=0)
                model.fit(X_tr, y_tr_cat, validation_data=(X_val, y_val_cat), epochs=20, batch_size=64, callbacks=[es], verbose=0)
                score = model.evaluate(X_val, y_val_cat, verbose=0)[1]  # accuracy
                fold_scores.append(score)
            avg_score = np.mean(fold_scores)
            if avg_score > best_score:
                best_score = avg_score
                best_params = {'units': units, 'dropout_rate': dropout_rate}
                best_model = build_lstm_model(input_shape, units=units, dropout_rate=dropout_rate, num_classes=num_classes)
                if num_classes > 2:
                    y_train_cat = to_categorical(y_train, num_classes=num_classes)
                else:
                    y_train_cat = y_train
                best_model.fit(X_train_scaled, y_train_cat, epochs=20, batch_size=64, verbose=0)

    print(f"\n=== Best Parameters for phase {phase}: {best_params} ===")

    # ==== Đánh giá mô hình gốc ====
    print(f"\n=== Evaluation for phase {phase} ===")
    
    # Chuyển y_test về dạng số nguyên (label), nếu cần
    if y_test.ndim > 1:
        y_test_label = np.argmax(y_test, axis=1)
    else:
        y_test_label = y_test
    
    evaluation_results = evaluate_model(
        best_model, X_train_scaled, y_train, X_test_scaled, y_test
    )
    train_results = evaluation_results['train']
    test_results = evaluation_results['test']
    
    for metric_name in ['f1', 'precision', 'recall', 'accuracy', 'auc']:
        print(f"Train {metric_name.capitalize()}: {train_results[metric_name]} | Test {metric_name.capitalize()}: {test_results[metric_name]}")
    
    # ==== Lưu kết quả dự đoán ====
    y_proba_test = best_model.predict(X_test_scaled)
    if y_proba_test.shape[1] == 1:
        y_pred = (y_proba_test > 0.5).astype(int).flatten()
    else:
        y_pred = np.argmax(y_proba_test, axis=1)
    
    results_df = pd.DataFrame(X_test.reshape(X_test.shape[0], -1))
    results_df['True_Label'] = y_test_label
    results_df['Predicted_Label'] = y_pred
    results_filename = f"test_results_{scale}_{metric}_{phase}.csv"
    results_df.to_csv(results_filename, index=False)
    print(f"Test predictions saved as {results_filename}")


    if num_classes == 2:
        label_names = ['A', 'B']
    else:
        label_names = ['A', 'B', 'C', 'D', 'E']
    
    print("\n=== Classification Report ===")
    print(classification_report(y_test_label, y_pred, target_names=label_names, digits=3))
        
    # ==== Calibrate model ====
    y_proba_train = best_model.predict(X_train_scaled)
    if y_train.ndim > 1:
        y_train_label = np.argmax(y_train, axis=1)
    else:
        y_train_label = y_train
    
    # Chọn LogisticRegression cho đúng binary/multi-class
    if y_proba_train.shape[1] == 1:
        calibrator = LogisticRegression()
    else:
        calibrator = LogisticRegression(multi_class="multinomial", solver="lbfgs", max_iter=1000)
    calibrator.fit(y_proba_train, y_train_label)
    
    class CalibratedLSTMWrapper:
        def predict(self, X):
            proba = best_model.predict(X)
            return calibrator.predict(proba)
    
    calibrate_train_model = CalibratedLSTMWrapper()
    
    print(f"\n=== Evaluation after calibration for phase {phase} ===")
    evaluation_results_cal = evaluate_model(
        calibrate_train_model, X_train_scaled, y_train_label, X_test_scaled, y_test_label
    )
    train_results_cal = evaluation_results_cal['train']
    test_results_cal = evaluation_results_cal['test']
    for metric_name in ['f1', 'precision', 'recall', 'accuracy', 'auc']:
        print(f"Train {metric_name.capitalize()} (calibrated): {train_results_cal[metric_name]} | Test {metric_name.capitalize()} (calibrated): {test_results_cal[metric_name]}")
    
    # ==== Lưu kết quả dự đoán sau calibrate ====
    y_calibrate_pred = calibrate_train_model.predict(X_test_scaled)
    results_df['Calibrated_Predicted_Label'] = y_calibrate_pred
    results_calibrate_filename = f"test_results_calibrate_{scale}_{metric}_{phase}.csv"
    results_df.to_csv(results_calibrate_filename, index=False)
    print(f"Calibrated predictions saved as {results_calibrate_filename}")
    
    if len(np.unique(y_train_label)) == 2:
        label_names = ['A', 'B']
    else:
        label_names = ['A', 'B', 'C', 'D', 'E']
    
    print("\n=== Classification Report after calibration ===")
    print(classification_report(y_test_label, y_calibrate_pred, target_names=label_names, digits=3))
    
    # ==== Nếu phase 1 hoặc 2 → chạy chọn feature ====
    if int(phase[-1]) <= 2:
        print(f"\n=== Permutation Importance for phase {phase} ===")
        perm_importance = plot_permutation_importance(best_model, X_train_scaled, y_train)
    
        print(f"\n=== Lasso Feature Selection for phase {phase} ===")
        lasso_importance = lasso_feature_selection(X_train_scaled, y_train)
        # Nếu muốn dùng Boruta bạn cần chuyển đổi model phù hợp hoặc dùng tree-based model


In [ ]:
for phase in ["phase1", "phase2", "phase3", "phase4"]:
    process_phase(phase, metric='f1_macro', timesteps=1)  # Chỉnh timesteps tuỳ ý


# 2. Minmax

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical


def build_lstm_model(input_shape, units=64, dropout_rate=0.2, num_classes=1):
    model = Sequential()
    model.add(LSTM(units, input_shape=input_shape, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='sigmoid' if num_classes == 1 else 'softmax'))
    model.compile(loss='binary_crossentropy' if num_classes == 1 else 'categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

def process_phase(phase, scale='minmax', metric='accuracy',timesteps=1):
    # Đọc dữ liệu
    X_train, y_train, X_test, y_test = load_data(phase, timesteps,one_hot=False)

    # if num_classes > 2:
    #     y_train_cat = to_categorical(y_train, num_classes=num_classes)
    # else:
    #     y_train_cat = y_train
    # Scale dữ liệu
    X_train_scaled, X_test_scaled = scale_data(X_train, X_test, scaler_type=scale)
    
    # Đảm bảo đúng shape cho LSTM: (samples, timesteps, features)
    if len(X_train_scaled.shape) == 2:
        # reshape nếu đầu vào chưa đủ 3 chiều
        X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, X_train_scaled.shape[1]))
        X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], timesteps, X_test_scaled.shape[1]))

    input_shape = (X_train_scaled.shape[1], X_train_scaled.shape[2])
    num_classes = 1 if len(np.unique(y_train)) == 2 else len(np.unique(y_train))

    # Thiết lập StratifiedKFold
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    best_score = -np.inf
    best_params = {}
    best_model = None

    # Grid search thủ công (do Keras không hợp với RandomizedSearchCV)
    for units in [32, 64]:
        for dropout_rate in [0.1, 0.2]:
            fold_scores = []
            for train_idx, val_idx in skf.split(X_train_scaled, y_train):
                X_tr, X_val = X_train_scaled[train_idx], X_train_scaled[val_idx]
                y_tr, y_val = y_train[train_idx], y_train[val_idx]
            
                if num_classes > 2:
                    y_tr_cat = to_categorical(y_tr, num_classes=num_classes)
                    y_val_cat = to_categorical(y_val, num_classes=num_classes)
                else:
                    y_tr_cat = y_tr
                    y_val_cat = y_val
            
                model = build_lstm_model(input_shape, units=units, dropout_rate=dropout_rate, num_classes=num_classes)
                es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=0)
                model.fit(X_tr, y_tr_cat, validation_data=(X_val, y_val_cat), epochs=20, batch_size=64, callbacks=[es], verbose=0)
                score = model.evaluate(X_val, y_val_cat, verbose=0)[1]  # accuracy
                fold_scores.append(score)
            avg_score = np.mean(fold_scores)
            if avg_score > best_score:
                best_score = avg_score
                best_params = {'units': units, 'dropout_rate': dropout_rate}
                best_model = build_lstm_model(input_shape, units=units, dropout_rate=dropout_rate, num_classes=num_classes)
                if num_classes > 2:
                    y_train_cat = to_categorical(y_train, num_classes=num_classes)
                else:
                    y_train_cat = y_train
                best_model.fit(X_train_scaled, y_train_cat, epochs=20, batch_size=64, verbose=0)

    print(f"\n=== Best Parameters for phase {phase}: {best_params} ===")

    # ==== Đánh giá mô hình gốc ====
    print(f"\n=== Evaluation for phase {phase} ===")
    
    # Chuyển y_test về dạng số nguyên (label), nếu cần
    if y_test.ndim > 1:
        y_test_label = np.argmax(y_test, axis=1)
    else:
        y_test_label = y_test
    
    evaluation_results = evaluate_model(
        best_model, X_train_scaled, y_train, X_test_scaled, y_test
    )
    train_results = evaluation_results['train']
    test_results = evaluation_results['test']
    
    for metric_name in ['f1', 'precision', 'recall', 'accuracy', 'auc']:
        print(f"Train {metric_name.capitalize()}: {train_results[metric_name]} | Test {metric_name.capitalize()}: {test_results[metric_name]}")
    
    # ==== Lưu kết quả dự đoán ====
    y_proba_test = best_model.predict(X_test_scaled)
    if y_proba_test.shape[1] == 1:
        y_pred = (y_proba_test > 0.5).astype(int).flatten()
    else:
        y_pred = np.argmax(y_proba_test, axis=1)
    
    results_df = pd.DataFrame(X_test.reshape(X_test.shape[0], -1))
    results_df['True_Label'] = y_test_label
    results_df['Predicted_Label'] = y_pred
    results_filename = f"test_results_{scale}_{metric}_{phase}.csv"
    results_df.to_csv(results_filename, index=False)
    print(f"Test predictions saved as {results_filename}")


    if num_classes == 2:
        label_names = ['A', 'B']
    else:
        label_names = ['A', 'B', 'C', 'D', 'E']
    
    print("\n=== Classification Report ===")
    print(classification_report(y_test_label, y_pred, target_names=label_names, digits=3))
        
    # ==== Calibrate model ====
    y_proba_train = best_model.predict(X_train_scaled)
    if y_train.ndim > 1:
        y_train_label = np.argmax(y_train, axis=1)
    else:
        y_train_label = y_train
    
    # Chọn LogisticRegression cho đúng binary/multi-class
    if y_proba_train.shape[1] == 1:
        calibrator = LogisticRegression()
    else:
        calibrator = LogisticRegression(multi_class="multinomial", solver="lbfgs", max_iter=1000)
    calibrator.fit(y_proba_train, y_train_label)
    
    class CalibratedLSTMWrapper:
        def predict(self, X):
            proba = best_model.predict(X)
            return calibrator.predict(proba)
    
    calibrate_train_model = CalibratedLSTMWrapper()
    
    print(f"\n=== Evaluation after calibration for phase {phase} ===")
    evaluation_results_cal = evaluate_model(
        calibrate_train_model, X_train_scaled, y_train_label, X_test_scaled, y_test_label
    )
    train_results_cal = evaluation_results_cal['train']
    test_results_cal = evaluation_results_cal['test']
    for metric_name in ['f1', 'precision', 'recall', 'accuracy', 'auc']:
        print(f"Train {metric_name.capitalize()} (calibrated): {train_results_cal[metric_name]} | Test {metric_name.capitalize()} (calibrated): {test_results_cal[metric_name]}")
    
    # ==== Lưu kết quả dự đoán sau calibrate ====
    y_calibrate_pred = calibrate_train_model.predict(X_test_scaled)
    results_df['Calibrated_Predicted_Label'] = y_calibrate_pred
    results_calibrate_filename = f"test_results_calibrate_{scale}_{metric}_{phase}.csv"
    results_df.to_csv(results_calibrate_filename, index=False)
    print(f"Calibrated predictions saved as {results_calibrate_filename}")
    
    if len(np.unique(y_train_label)) == 2:
        label_names = ['A', 'B']
    else:
        label_names = ['A', 'B', 'C', 'D', 'E']
    
    print("\n=== Classification Report after calibration ===")
    print(classification_report(y_test_label, y_calibrate_pred, target_names=label_names, digits=3))
    
    # ==== Nếu phase 1 hoặc 2 → chạy chọn feature ====
    if int(phase[-1]) <= 2:
        print(f"\n=== Permutation Importance for phase {phase} ===")
        perm_importance = plot_permutation_importance(best_model, X_train_scaled, y_train)
    
        print(f"\n=== Lasso Feature Selection for phase {phase} ===")
        lasso_importance = lasso_feature_selection(X_train_scaled, y_train)
        # Nếu muốn dùng Boruta bạn cần chuyển đổi model phù hợp hoặc dùng tree-based model


In [ ]:
for phase in ["phase1", "phase2", "phase3", "phase4"]:
    process_phase(phase, metric='f1_macro', timesteps=1)  # Chỉnh timesteps tuỳ ý


# 3. Robust

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical


def build_lstm_model(input_shape, units=64, dropout_rate=0.2, num_classes=1):
    model = Sequential()
    model.add(LSTM(units, input_shape=input_shape, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='sigmoid' if num_classes == 1 else 'softmax'))
    model.compile(loss='binary_crossentropy' if num_classes == 1 else 'categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

def process_phase(phase, scale='robust', metric='accuracy',timesteps=1):
    # Đọc dữ liệu
    X_train, y_train, X_test, y_test = load_data(phase, timesteps,one_hot=False)

    # if num_classes > 2:
    #     y_train_cat = to_categorical(y_train, num_classes=num_classes)
    # else:
    #     y_train_cat = y_train
    # Scale dữ liệu
    X_train_scaled, X_test_scaled = scale_data(X_train, X_test, scaler_type=scale)
    
    # Đảm bảo đúng shape cho LSTM: (samples, timesteps, features)
    if len(X_train_scaled.shape) == 2:
        # reshape nếu đầu vào chưa đủ 3 chiều
        X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, X_train_scaled.shape[1]))
        X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], timesteps, X_test_scaled.shape[1]))

    input_shape = (X_train_scaled.shape[1], X_train_scaled.shape[2])
    num_classes = 1 if len(np.unique(y_train)) == 2 else len(np.unique(y_train))

    # Thiết lập StratifiedKFold
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    best_score = -np.inf
    best_params = {}
    best_model = None

    # Grid search thủ công (do Keras không hợp với RandomizedSearchCV)
    for units in [32, 64]:
        for dropout_rate in [0.1, 0.2]:
            fold_scores = []
            for train_idx, val_idx in skf.split(X_train_scaled, y_train):
                X_tr, X_val = X_train_scaled[train_idx], X_train_scaled[val_idx]
                y_tr, y_val = y_train[train_idx], y_train[val_idx]
            
                if num_classes > 2:
                    y_tr_cat = to_categorical(y_tr, num_classes=num_classes)
                    y_val_cat = to_categorical(y_val, num_classes=num_classes)
                else:
                    y_tr_cat = y_tr
                    y_val_cat = y_val
            
                model = build_lstm_model(input_shape, units=units, dropout_rate=dropout_rate, num_classes=num_classes)
                es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=0)
                model.fit(X_tr, y_tr_cat, validation_data=(X_val, y_val_cat), epochs=20, batch_size=64, callbacks=[es], verbose=0)
                score = model.evaluate(X_val, y_val_cat, verbose=0)[1]  # accuracy
                fold_scores.append(score)
            avg_score = np.mean(fold_scores)
            if avg_score > best_score:
                best_score = avg_score
                best_params = {'units': units, 'dropout_rate': dropout_rate}
                best_model = build_lstm_model(input_shape, units=units, dropout_rate=dropout_rate, num_classes=num_classes)
                if num_classes > 2:
                    y_train_cat = to_categorical(y_train, num_classes=num_classes)
                else:
                    y_train_cat = y_train
                best_model.fit(X_train_scaled, y_train_cat, epochs=20, batch_size=64, verbose=0)

    print(f"\n=== Best Parameters for phase {phase}: {best_params} ===")

    # ==== Đánh giá mô hình gốc ====
    print(f"\n=== Evaluation for phase {phase} ===")
    
    # Chuyển y_test về dạng số nguyên (label), nếu cần
    if y_test.ndim > 1:
        y_test_label = np.argmax(y_test, axis=1)
    else:
        y_test_label = y_test
    
    evaluation_results = evaluate_model(
        best_model, X_train_scaled, y_train, X_test_scaled, y_test
    )
    train_results = evaluation_results['train']
    test_results = evaluation_results['test']
    
    for metric_name in ['f1', 'precision', 'recall', 'accuracy', 'auc']:
        print(f"Train {metric_name.capitalize()}: {train_results[metric_name]} | Test {metric_name.capitalize()}: {test_results[metric_name]}")
    
    # ==== Lưu kết quả dự đoán ====
    y_proba_test = best_model.predict(X_test_scaled)
    if y_proba_test.shape[1] == 1:
        y_pred = (y_proba_test > 0.5).astype(int).flatten()
    else:
        y_pred = np.argmax(y_proba_test, axis=1)
    
    results_df = pd.DataFrame(X_test.reshape(X_test.shape[0], -1))
    results_df['True_Label'] = y_test_label
    results_df['Predicted_Label'] = y_pred
    results_filename = f"test_results_{scale}_{metric}_{phase}.csv"
    results_df.to_csv(results_filename, index=False)
    print(f"Test predictions saved as {results_filename}")


    if num_classes == 2:
        label_names = ['A', 'B']
    else:
        label_names = ['A', 'B', 'C', 'D', 'E']
    
    print("\n=== Classification Report ===")
    print(classification_report(y_test_label, y_pred, target_names=label_names, digits=3))
        
    # ==== Calibrate model ====
    y_proba_train = best_model.predict(X_train_scaled)
    if y_train.ndim > 1:
        y_train_label = np.argmax(y_train, axis=1)
    else:
        y_train_label = y_train
    
    # Chọn LogisticRegression cho đúng binary/multi-class
    if y_proba_train.shape[1] == 1:
        calibrator = LogisticRegression()
    else:
        calibrator = LogisticRegression(multi_class="multinomial", solver="lbfgs", max_iter=1000)
    calibrator.fit(y_proba_train, y_train_label)
    
    class CalibratedLSTMWrapper:
        def predict(self, X):
            proba = best_model.predict(X)
            return calibrator.predict(proba)
    
    calibrate_train_model = CalibratedLSTMWrapper()
    
    print(f"\n=== Evaluation after calibration for phase {phase} ===")
    evaluation_results_cal = evaluate_model(
        calibrate_train_model, X_train_scaled, y_train_label, X_test_scaled, y_test_label
    )
    train_results_cal = evaluation_results_cal['train']
    test_results_cal = evaluation_results_cal['test']
    for metric_name in ['f1', 'precision', 'recall', 'accuracy', 'auc']:
        print(f"Train {metric_name.capitalize()} (calibrated): {train_results_cal[metric_name]} | Test {metric_name.capitalize()} (calibrated): {test_results_cal[metric_name]}")
    
    # ==== Lưu kết quả dự đoán sau calibrate ====
    y_calibrate_pred = calibrate_train_model.predict(X_test_scaled)
    results_df['Calibrated_Predicted_Label'] = y_calibrate_pred
    results_calibrate_filename = f"test_results_calibrate_{scale}_{metric}_{phase}.csv"
    results_df.to_csv(results_calibrate_filename, index=False)
    print(f"Calibrated predictions saved as {results_calibrate_filename}")
    
    if len(np.unique(y_train_label)) == 2:
        label_names = ['A', 'B']
    else:
        label_names = ['A', 'B', 'C', 'D', 'E']
    
    print("\n=== Classification Report after calibration ===")
    print(classification_report(y_test_label, y_calibrate_pred, target_names=label_names, digits=3))
    
    # ==== Nếu phase 1 hoặc 2 → chạy chọn feature ====
    if int(phase[-1]) <= 2:
        print(f"\n=== Permutation Importance for phase {phase} ===")
        perm_importance = plot_permutation_importance(best_model, X_train_scaled, y_train)
    
        print(f"\n=== Lasso Feature Selection for phase {phase} ===")
        lasso_importance = lasso_feature_selection(X_train_scaled, y_train)
        # Nếu muốn dùng Boruta bạn cần chuyển đổi model phù hợp hoặc dùng tree-based model


In [ ]:
for phase in ["phase1", "phase2", "phase3", "phase4"]:
    process_phase(phase, metric='f1_macro', timesteps=1)  # Chỉnh timesteps tuỳ ý


# 4. Log

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical


def build_lstm_model(input_shape, units=64, dropout_rate=0.2, num_classes=1):
    model = Sequential()
    model.add(LSTM(units, input_shape=input_shape, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='sigmoid' if num_classes == 1 else 'softmax'))
    model.compile(loss='binary_crossentropy' if num_classes == 1 else 'categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

def process_phase(phase, scale='log', metric='accuracy',timesteps=1):
    # Đọc dữ liệu
    X_train, y_train, X_test, y_test = load_data(phase, timesteps,one_hot=False)

    # if num_classes > 2:
    #     y_train_cat = to_categorical(y_train, num_classes=num_classes)
    # else:
    #     y_train_cat = y_train
    # Scale dữ liệu
    X_train_scaled, X_test_scaled = scale_data(X_train, X_test, scaler_type=scale)
    
    # Đảm bảo đúng shape cho LSTM: (samples, timesteps, features)
    if len(X_train_scaled.shape) == 2:
        # reshape nếu đầu vào chưa đủ 3 chiều
        X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, X_train_scaled.shape[1]))
        X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], timesteps, X_test_scaled.shape[1]))

    input_shape = (X_train_scaled.shape[1], X_train_scaled.shape[2])
    num_classes = 1 if len(np.unique(y_train)) == 2 else len(np.unique(y_train))

    # Thiết lập StratifiedKFold
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    best_score = -np.inf
    best_params = {}
    best_model = None

    # Grid search thủ công (do Keras không hợp với RandomizedSearchCV)
    for units in [32, 64]:
        for dropout_rate in [0.1, 0.2]:
            fold_scores = []
            for train_idx, val_idx in skf.split(X_train_scaled, y_train):
                X_tr, X_val = X_train_scaled[train_idx], X_train_scaled[val_idx]
                y_tr, y_val = y_train[train_idx], y_train[val_idx]
            
                if num_classes > 2:
                    y_tr_cat = to_categorical(y_tr, num_classes=num_classes)
                    y_val_cat = to_categorical(y_val, num_classes=num_classes)
                else:
                    y_tr_cat = y_tr
                    y_val_cat = y_val
            
                model = build_lstm_model(input_shape, units=units, dropout_rate=dropout_rate, num_classes=num_classes)
                es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=0)
                model.fit(X_tr, y_tr_cat, validation_data=(X_val, y_val_cat), epochs=20, batch_size=64, callbacks=[es], verbose=0)
                score = model.evaluate(X_val, y_val_cat, verbose=0)[1]  # accuracy
                fold_scores.append(score)
            avg_score = np.mean(fold_scores)
            if avg_score > best_score:
                best_score = avg_score
                best_params = {'units': units, 'dropout_rate': dropout_rate}
                best_model = build_lstm_model(input_shape, units=units, dropout_rate=dropout_rate, num_classes=num_classes)
                if num_classes > 2:
                    y_train_cat = to_categorical(y_train, num_classes=num_classes)
                else:
                    y_train_cat = y_train
                best_model.fit(X_train_scaled, y_train_cat, epochs=20, batch_size=64, verbose=0)

    print(f"\n=== Best Parameters for phase {phase}: {best_params} ===")

    # ==== Đánh giá mô hình gốc ====
    print(f"\n=== Evaluation for phase {phase} ===")
    
    # Chuyển y_test về dạng số nguyên (label), nếu cần
    if y_test.ndim > 1:
        y_test_label = np.argmax(y_test, axis=1)
    else:
        y_test_label = y_test
    
    evaluation_results = evaluate_model(
        best_model, X_train_scaled, y_train, X_test_scaled, y_test
    )
    train_results = evaluation_results['train']
    test_results = evaluation_results['test']
    
    for metric_name in ['f1', 'precision', 'recall', 'accuracy', 'auc']:
        print(f"Train {metric_name.capitalize()}: {train_results[metric_name]} | Test {metric_name.capitalize()}: {test_results[metric_name]}")
    
    # ==== Lưu kết quả dự đoán ====
    y_proba_test = best_model.predict(X_test_scaled)
    if y_proba_test.shape[1] == 1:
        y_pred = (y_proba_test > 0.5).astype(int).flatten()
    else:
        y_pred = np.argmax(y_proba_test, axis=1)
    
    results_df = pd.DataFrame(X_test.reshape(X_test.shape[0], -1))
    results_df['True_Label'] = y_test_label
    results_df['Predicted_Label'] = y_pred
    results_filename = f"test_results_{scale}_{metric}_{phase}.csv"
    results_df.to_csv(results_filename, index=False)
    print(f"Test predictions saved as {results_filename}")


    if num_classes == 2:
        label_names = ['A', 'B']
    else:
        label_names = ['A', 'B', 'C', 'D', 'E']
    
    print("\n=== Classification Report ===")
    print(classification_report(y_test_label, y_pred, target_names=label_names, digits=3))
        
    # ==== Calibrate model ====
    y_proba_train = best_model.predict(X_train_scaled)
    if y_train.ndim > 1:
        y_train_label = np.argmax(y_train, axis=1)
    else:
        y_train_label = y_train
    
    # Chọn LogisticRegression cho đúng binary/multi-class
    if y_proba_train.shape[1] == 1:
        calibrator = LogisticRegression()
    else:
        calibrator = LogisticRegression(multi_class="multinomial", solver="lbfgs", max_iter=1000)
    calibrator.fit(y_proba_train, y_train_label)
    
    class CalibratedLSTMWrapper:
        def predict(self, X):
            proba = best_model.predict(X)
            return calibrator.predict(proba)
    
    calibrate_train_model = CalibratedLSTMWrapper()
    
    print(f"\n=== Evaluation after calibration for phase {phase} ===")
    evaluation_results_cal = evaluate_model(
        calibrate_train_model, X_train_scaled, y_train_label, X_test_scaled, y_test_label
    )
    train_results_cal = evaluation_results_cal['train']
    test_results_cal = evaluation_results_cal['test']
    for metric_name in ['f1', 'precision', 'recall', 'accuracy', 'auc']:
        print(f"Train {metric_name.capitalize()} (calibrated): {train_results_cal[metric_name]} | Test {metric_name.capitalize()} (calibrated): {test_results_cal[metric_name]}")
    
    # ==== Lưu kết quả dự đoán sau calibrate ====
    y_calibrate_pred = calibrate_train_model.predict(X_test_scaled)
    results_df['Calibrated_Predicted_Label'] = y_calibrate_pred
    results_calibrate_filename = f"test_results_calibrate_{scale}_{metric}_{phase}.csv"
    results_df.to_csv(results_calibrate_filename, index=False)
    print(f"Calibrated predictions saved as {results_calibrate_filename}")
    
    if len(np.unique(y_train_label)) == 2:
        label_names = ['A', 'B']
    else:
        label_names = ['A', 'B', 'C', 'D', 'E']
    
    print("\n=== Classification Report after calibration ===")
    print(classification_report(y_test_label, y_calibrate_pred, target_names=label_names, digits=3))
    
    # ==== Nếu phase 1 hoặc 2 → chạy chọn feature ====
    if int(phase[-1]) <= 2:
        print(f"\n=== Permutation Importance for phase {phase} ===")
        perm_importance = plot_permutation_importance(best_model, X_train_scaled, y_train)
    
        print(f"\n=== Lasso Feature Selection for phase {phase} ===")
        lasso_importance = lasso_feature_selection(X_train_scaled, y_train)
        # Nếu muốn dùng Boruta bạn cần chuyển đổi model phù hợp hoặc dùng tree-based model


In [ ]:
for phase in ["phase1", "phase2", "phase3", "phase4"]:
    process_phase(phase, metric='f1_macro', timesteps=1)  # Chỉnh timesteps tuỳ ý
